In [1]:
import pandas as pd
import pickle
import pprint
import random

In [2]:
%%time

individual_provider_payment_df_clean = pd.read_pickle("../data/individual_provider_payment.pkl")

Wall time: 34.5 s


In [5]:
# creating total revenue column

individual_provider_payment_df_clean_no_office['Total Revenue'] = individual_provider_payment_df_clean_no_office['Average Medicare Allowed Amount'] * individual_provider_payment_df_clean_no_office['Number of Services']

In [3]:
individual_provider_payment_df_clean.describe()

,National Provider Identifier,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services,Average Medicare Allowed Amount,Average Medicare Payment Amount
count,9.416125e+06,9.416125e+06,9.416125e+06,9.416125e+06,9.416125e+06,9.416125e+06
mean,1.499785e+09,1.983183e+02,7.084164e+01,1.151297e+02,9.735926e+01,7.392625e+01
std,2.877289e+08,2.351712e+03,1.442648e+02,2.783331e+02,2.186003e+02,1.705221e+02
min,1.003000e+09,2.400000e+00,1.100000e+01,1.100000e+01,6.035380e-05,0.000000e+00
25%,1.255302e+09,2.000000e+01,1.700000e+01,2.000000e+01,2.501000e+01,1.967050e+01
50%,1.497969e+09,4.300000e+01,3.200000e+01,4.000000e+01,6.639000e+01,4.745194e+01
75%,1.740681e+09,1.160000e+02,7.300000e+01,1.040000e+02,1.131400e+02,8.456263e+01
max,1.993000e+09,3.301134e+06,9.532700e+04,1.758700e+05,3.986417e+04,3.125351e+04


In [4]:
individual_provider_payment_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9416125 entries, 0 to 9416124
Data columns (total 21 columns):
 #   Column                                                    Dtype  
---  ------                                                    -----  
 0   National Provider Identifier                              int64  
 1   Last Name/Organization Name of the Provider               object 
 2   First Name of the Provider                                object 
 3   Credentials of the Provider                               object 
 4   Entity Type of the Provider                               object 
 5   Street Address 1 of the Provider                          object 
 6   City of the Provider                                      object 
 7   Zip Code of the Provider                                  object 
 8   State Code of the Provider                                object 
 9   Country Code of the Provider                              object 
 10  Provider Type                 

In [5]:
individual_provider_payment_df_clean['Provider Type'].value_counts()

Diagnostic Radiology                   1241400
Internal Medicine                      1118171
Family Practice                         969268
Nurse Practitioner                      560219
Cardiology                              445088
                                        ...   
Ambulance Service Provider                  42
Unknown Supplier/Provider Specialty         17
All Other Suppliers                         11
Medical Toxicology                           2
Slide Preparation Facility                   2
Name: Provider Type, Length: 88, dtype: int64

In [6]:
individual_provider_payment_df_clean['HCPCS Description'].value_counts()

Established patient office or other outpatient visit, typically 15 minutes    457835
Established patient office or other outpatient, visit typically 25 minutes    428262
New patient office or other outpatient visit, typically 45 minutes            179569
New patient office or other outpatient visit, typically 30 minutes            178057
Subsequent hospital inpatient care, typically 25 minutes per day              175335
                                                                               ...  
Drainage of abscess or blood accumulation at forearm and/or wrist                  1
Repair of windpipe cartilage by insertion of splint or graft                       1
Removal of nerve of foot muscle                                                    1
Bone graft of pelvic bone with microvascular connection                            1
Removal of bladder, urinary ducts (ureters)                                        1
Name: HCPCS Description, Length: 5171, dtype: int64

In [7]:
individual_provider_payment_df_clean['HCPCS Code'].value_counts()

99213    457835
99214    428262
99204    179569
99203    178057
99232    175335
          ...  
67120         1
80190         1
44314         1
96420         1
86622         1
Name: HCPCS Code, Length: 5702, dtype: int64

In [60]:
# informational to make sure my row drops will be dropping what I expect it to drop
individual_provider_payment_df_clean.loc[individual_provider_payment_df_clean['HCPCS Code'].str.contains('992..|993..', regex=True)]['HCPCS Code'].sort_values()

4075581    99201
4887937    99201
529176     99201
1642193    99201
7929017    99201
           ...  
8736400    99359
8554568    99359
9286250    99359
5783575    99359
8609130    99359
Name: HCPCS Code, Length: 2957859, dtype: object

In [3]:
# remove patient office visits
individual_provider_payment_df_clean_no_office = individual_provider_payment_df_clean.loc[~individual_provider_payment_df_clean['HCPCS Code'].str.contains('992..|993..', regex=True)]

In [4]:
individual_provider_payment_df_clean_no_office = individual_provider_payment_df_clean_no_office.reset_index(drop=True)

In [14]:
# creating groupings for codes

HCPCS_Grouped_Code = []

for value in individual_provider_payment_df_clean_no_office['HCPCS Code']:
    if value.endswith("T"):
        HCPCS_Grouped_Code.append("0000T")
    else:
        HCPCS_Grouped_Code.append(value[0:3] + "00")
        
     

In [17]:
individual_provider_payment_df_clean_no_office['HCPCS Grouped Code'] = HCPCS_Grouped_Code 

In [18]:
# write file without 99200 and 99300 to csv
individual_provider_payment_df_clean_no_office.to_csv("../data/provider_payments_exclude_99200_and_99300.csv", index=False)

In [130]:
provider_type_sum_of_services = individual_provider_payment_df_clean_no_office.groupby('Provider Type', as_index=False)['Number of Services'].sum()

provider_type_services = individual_provider_payment_df_clean_no_office.groupby(['Provider Type','HCPCS Code'], as_index=False)['Number of Services'].sum()

provider_type_grouped_services = individual_provider_payment_df_clean_no_office.groupby(['Provider Type','HCPCS Grouped Code'], as_index=False)['Number of Services'].sum()

print(provider_type_sum_of_services)

                                       Provider Type  Number of Services
0                                 Addiction Medicine             38282.0
1   Advanced Heart Failure and Transplant Cardiology              4219.0
2                                All Other Suppliers              1109.0
3                                Allergy/ Immunology          15512077.9
4                         Ambulance Service Provider            233194.9
5                                     Anesthesiology          12099052.1
6                           Anesthesiology Assistant            119622.0
7                                        Audiologist           1967592.0
8                                    Cardiac Surgery            482515.0
9                                         Cardiology          47650180.9
10               Certified Clinical Nurse Specialist            295045.5
11                           Certified Nurse Midwife             17657.0
12     Certified Registered Nurse Anesthetist (CRNA

In [124]:
print(provider_type_services.head())
print(provider_type_sum_of_services.head())
print(provider_type_grouped_services.head())

        Provider Type HCPCS Code  Number of Services
0  Addiction Medicine      36415               595.0
1  Addiction Medicine      36556                12.0
2  Addiction Medicine      36620                13.0
3  Addiction Medicine      70100               590.0
4  Addiction Medicine      70310               590.0
                                      Provider Type  Number of Services
0                                Addiction Medicine             38282.0
1  Advanced Heart Failure and Transplant Cardiology              4219.0
2                               All Other Suppliers              1109.0
3                               Allergy/ Immunology          15512077.9
4                        Ambulance Service Provider            233194.9
        Provider Type HCPCS Grouped Code  Number of Services
0  Addiction Medicine              36400               595.0
1  Addiction Medicine              36500                12.0
2  Addiction Medicine              36600                13.0
3  Add

In [126]:
provider_type_sum_of_services_df = pd.DataFrame({"Provider Type": provider_type_sum_of_services['Provider Type'], "total_count": provider_type_sum_of_services["Number of Services"]})

print(provider_type_sum_of_services_df.head())

provider_type_services_df = pd.DataFrame({ "Provider Type":provider_type_services["Provider Type"], "HCPCS Code":provider_type_services["HCPCS Code"], "service_count":provider_type_services["Number of Services"]})
# provider_type_services_df = provider_type_services_df.reset_index(level='HCPCS Code')

provider_type_grouped_services_df = pd.DataFrame({"Provider Type": provider_type_grouped_services['Provider Type'], "HCPCS Grouped Code":provider_type_grouped_services["HCPCS Grouped Code"], "grouped_service_count":provider_type_grouped_services["Number of Services"]})
# provider_type_grouped_services_df = provider_type_grouped_services_df.reset_index(level='HCPCS Grouped Code')

print(provider_type_grouped_services_df.head())

                                      Provider Type  total_count
0                                Addiction Medicine      38282.0
1  Advanced Heart Failure and Transplant Cardiology       4219.0
2                               All Other Suppliers       1109.0
3                               Allergy/ Immunology   15512077.9
4                        Ambulance Service Provider     233194.9
        Provider Type HCPCS Grouped Code  grouped_service_count
0  Addiction Medicine              36400                  595.0
1  Addiction Medicine              36500                   12.0
2  Addiction Medicine              36600                   13.0
3  Addiction Medicine              70100                  590.0
4  Addiction Medicine              70300                 1081.0


In [55]:
provider_type_sum_of_services.index

Index(['Addiction Medicine',
       'Advanced Heart Failure and Transplant Cardiology',
       'All Other Suppliers', 'Allergy/ Immunology',
       'Ambulance Service Provider', 'Anesthesiology',
       'Anesthesiology Assistant', 'Audiologist', 'Cardiac Surgery',
       'Cardiology', 'Certified Clinical Nurse Specialist',
       'Certified Nurse Midwife',
       'Certified Registered Nurse Anesthetist (CRNA)', 'Chiropractic',
       'Clinic or Group Practice', 'Clinical Cardiac Electrophysiology',
       'Clinical Laboratory', 'Colorectal Surgery (Proctology)',
       'Critical Care (Intensivists)', 'Dentist', 'Dermatology',
       'Diagnostic Radiology', 'Emergency Medicine', 'Endocrinology',
       'Family Practice', 'Gastroenterology', 'General Practice',
       'General Surgery', 'Geriatric Medicine', 'Geriatric Psychiatry',
       'Gynecological Oncology', 'Hand Surgery', 'Hematology',
       'Hematology-Oncology', 'Hospice and Palliative Care', 'Hospitalist',
       'Independent

In [127]:
individual_provider_payment_df_clean_no_office_merged = pd.merge(individual_provider_payment_df_clean_no_office, provider_type_services_df, how="left", on=['Provider Type','HCPCS Code'])
individual_provider_payment_df_clean_no_office_merged = pd.merge(individual_provider_payment_df_clean_no_office_merged, provider_type_sum_of_services_df, how="left", on='Provider Type')
individual_provider_payment_df_clean_no_office_merged = pd.merge(individual_provider_payment_df_clean_no_office_merged, provider_type_grouped_services_df, how="left", on=['Provider Type','HCPCS Grouped Code'])
individual_provider_payment_df_clean_no_office_merged['service_ratio'] = individual_provider_payment_df_clean_no_office_merged.service_count/individual_provider_payment_df_clean_no_office_merged.total_count
individual_provider_payment_df_clean_no_office_merged['grouped_service_ratio'] = individual_provider_payment_df_clean_no_office_merged.grouped_service_count/individual_provider_payment_df_clean_no_office_merged.total_count

individual_provider_payment_df_clean_no_office_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6458266 entries, 0 to 6458265
Data columns (total 28 columns):
 #   Column                                                    Dtype  
---  ------                                                    -----  
 0   National Provider Identifier                              int64  
 1   Last Name/Organization Name of the Provider               object 
 2   First Name of the Provider                                object 
 3   Credentials of the Provider                               object 
 4   Entity Type of the Provider                               object 
 5   Street Address 1 of the Provider                          object 
 6   City of the Provider                                      object 
 7   Zip Code of the Provider                                  object 
 8   State Code of the Provider                                object 
 9   Country Code of the Provider                              object 
 10  Provider Type                 

In [97]:
individual_provider_payment_df_clean_no_office_merged.service_count

0            759792.0
1          11238971.8
2            872574.0
3           1020068.1
4            116120.0
              ...    
6458261      528132.0
6458262      566218.0
6458263      566218.0
6458264      858304.0
6458265      858304.0
Name: service_count, Length: 6458266, dtype: float64

In [104]:
pd.options.display.max_rows = 100

individual_provider_payment_df_clean_no_office_merged[["Provider Type", "total_count"]]

,Provider Type,total_count
0,Pathology,29516564.8
1,Pathology,29516564.8
2,Pathology,29516564.8
3,Pathology,29516564.8
4,Pathology,29516564.8
...,...,...
6458261,Otolaryngology,8748779.9
6458262,Otolaryngology,8748779.9
6458263,Otolaryngology,8748779.9
6458264,Otolaryngology,8748779.9


In [128]:
# write file without 99200 and 99300; with service_counts and total_count and ratio to csv
individual_provider_payment_df_clean_no_office_merged.to_csv("../data/final_data_file.csv", index=False)

In [68]:
individual_provider_payment_df_clean_no_office_merged.shape

(6458266, 26)

In [18]:
provider_no_office

Average Medicare Allowed Amount  \
Provider Type      HCPCS Code HCPCS Description                                                                     
Addiction Medicine 36415      Insertion of needle into vein for collection of...                         3.000000   
                   36556      Insertion of central venous catheter for infusi...                       112.510000   
                   36620      Insertion of arterial catheter for blood sampli...                        47.990000   
                   70100      X-ray of mandible, less than 4 views                                      12.624219   
                   70310      X-ray of teeth, less than full mouth                                      12.237704   
...                                                                                                           ...   
Vascular Surgery   Q4131      Epifix or epicord, per square centimeter                                 161.975351   
                   Q4148      Neox 1k, per square centimeter                                           250.330000   
                   Q9965      Low osmolar contrast material, 100-199 mg/ml io...                         0.834526   
                   Q9966      Low osmolar contrast material, 200-299 mg/ml io...                         0.246482   
                   Q9967      Low osmolar contrast material, 300-399 mg/ml io...                         0.124234   

                                                                                  Average Medicare Payment Amount  \
Provider Type      HCPCS Code HCPCS Description                                                                     
Addiction Medicine 36415      Insertion of needle into vein for collection of...                         2.933529   
                   36556      Insertion of central venous catheter for infusi...                        88.210000   
                   36620      Insertion of arterial catheter for blood sampli...                        37.620000   
                   70100      X-ray of mandible, less than 4 views                                       9.532713   
                   70310      X-ray of teeth, less than full mouth                                       9.257998   
...                                                                                                           ...   
Vascular Surgery   Q4131      Epifix or epicord, per square centimeter                                 126.948602   
                   Q4148      Neox 1k, per square centimeter                                           196.258910   
                   Q9965      Low osmolar contrast material, 100-199 mg/ml io...                         0.654269   
                   Q9966      Low osmolar contrast material, 200-299 mg/ml io...                         0.192448   
                   Q9967      Low osmolar contrast material, 300-399 mg/ml io...                         0.096476   

                                                                                  National Provider Identifier  \
Provider Type      HCPCS Code HCPCS Description                                                                  
Addiction Medicine 36415      Insertion of needle into vein for collection of...                  1.443171e+09   
                   36556      Insertion of central venous catheter for infusi...                  1.770876e+09   
                   36620      Insertion of arterial catheter for blood sampli...                  1.770876e+09   
                   70100      X-ray of mandible, less than 4 views                                1.306066e+09   
                   70310      X-ray of teeth, less than full mouth                                1.306066e+09   
...                                                                                                        ...   
Vascular Surgery   Q4131      Epifix or epicord, per square centimeter                            1.543785e+09   
            

In [31]:
names = ['Amanda', 'Ness', 'Cat', 'Ben', 'Jacob', 'Sophia']


In [32]:
generator = names.pop(random.randrange(0,len(names),1))
print(generator)

Amanda
